In [ ]:

import pandas as pd
from connection import connect
from utils.days_and_moths import english_days, spanish_days, french_days, english_months, spanish_months, french_months

In [ ]:
SCHEMA='production'
co_oltp,etl_conn,etl_conn_or=connect(SCHEMA)

In [ ]:
dim_fecha = pd.DataFrame({
    "date": pd.date_range(start='1/1/2005', end='31/12/2014', freq='D')
})

In [ ]:
dim_fecha['day_number_of_week'] = ((dim_fecha['date'].dt.day_of_week + 1) % 7) + 1
dim_fecha['day_number_of_month']=dim_fecha['date'].dt.day
dim_fecha['day_number_of_year']=dim_fecha['date'].dt.day_of_year
dim_fecha['week_number_of_year'] = dim_fecha['date'].apply(
    lambda d: ((d - pd.Timestamp(d.year, 1, 1)).days + pd.Timestamp(d.year, 1, 1).weekday() + 1) // 7 + 1
)
dim_fecha['month_number_of_year']=dim_fecha['date'].dt.month
dim_fecha['calendar_quarter']=dim_fecha['date'].dt.quarter
dim_fecha['calendar_year']=dim_fecha['date'].dt.year
dim_fecha['calendar_semester']=(((dim_fecha['date'].dt.month-1)//6)+1)
dim_fecha['fiscal_year']=dim_fecha['date'].apply(lambda d: d.year if d.month <=7 else d.year+1)
dim_fecha['fiscal_month']=dim_fecha['date'].apply(lambda d:((d.month)-7)%12+1)
dim_fecha['fiscal_quarter']=((dim_fecha['fiscal_month']-1)//3) + 1
dim_fecha['fiscal_semester']=((dim_fecha['fiscal_month']-1)//6) + 1

dim_fecha.drop('fiscal_month',inplace=True,axis=1)

#languages

dim_fecha['english_day_name_of_week'] = dim_fecha['day_number_of_week'].map(english_days)
dim_fecha['spanish_day_name_of_week']=dim_fecha['day_number_of_week'].map(spanish_days)
dim_fecha['french_day_name_of_week']=dim_fecha['day_number_of_week'].map(french_days)

dim_fecha['english_month_name']=dim_fecha['month_number_of_year'].map(english_months)
dim_fecha['spanish_month_name']=dim_fecha['month_number_of_year'].map(spanish_months)
dim_fecha['french_month_name']=dim_fecha['month_number_of_year'].map(french_months)


dim_fecha


In [ ]:
dim_fecha['date_key']=dim_fecha['date'].dt.strftime('%Y%m%d').astype(int)


dim_fecha.rename(columns={'date':'full_date_alternate_key'},inplace=True)


dim_fecha

## Load

In [20]:
dim_fecha.to_sql('dim_date',etl_conn,if_exists='append',index=False)

652